In [332]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
from datetime import datetime

In [333]:
data = pd.read_csv("/Users/yuxiding/Desktop/dataset.s010", sep="\t")

In [334]:
diabetic = data[data['subjectId'] == "1636-69-091"]
prediabetic = data[data['subjectId'] == "1636-69-032"]
non_diabetic = data[data['subjectId'] == "1636-69-028"]

In [358]:
# REFERENCES: https://care.diabetesjournals.org/content/40/12/1631

def auc(x, unit = 'mg'):
    if unit == 'mg':
        gv = (x['GlucoseValue']).astype(float)
    elif unit == 'mmol':
        gv = (18*x['GlucoseValue']).astype()
    else:
        print('Unit should either be mg or mmol')
    

    full_day = list((splitDay(x)).items())[1:5]
    a = hourAve(full_day[0][1])[0]
    
    auc_list = []
    auc_list_daytime = []
    auc_list_nighttime = []
    for i in range(len(full_day)):
        auc_per_day = []
        hour_ave_gv = hourAve(full_day[i][1])[0]
        
        for idx in range(1, len(hour_ave_gv)):
            trapezoid = (hour_ave_gv['GlucoseValue'].iloc[idx] + hour_ave_gv['GlucoseValue'].iloc[idx-1]) * 1 * 0.5
            auc_per_day.append(trapezoid)
        
        auc_list.append(np.average(auc_per_day))
        auc_list_daytime.append(np.average(auc_per_day[6:]))
        auc_list_nighttime.append(np.average(auc_per_day[0:6]))

    auc_24h = np.average(auc_list)
    auc_day = np.average(auc_list_daytime)
    auc_night = np.average(auc_list_nighttime)
        
    return auc_24h, auc_day, auc_night

In [359]:
auc(prediabetic)

(106.96603260869566, 108.50980392156863, 102.59201388888889)

In [26]:
# REFERENCES: https://www.liebertpub.com/doi/pdf/10.1089/dia.2008.0132
# Index of Glycemic Control
def igc(x, unit = 'mg', lltr = 80, ultr = 140, a = 1.1, b = 2.0, c = 30, d = 30):
    
    if unit == 'mg':
        gv = (x['GlucoseValue']).astype(float)
    elif unit == 'mmol':
        gv = (18*x['GlucoseValue']).astype(float)
    else:
        print('Unit should either be mg or mmol')
    
    lower = gv[gv < 90]
    upper = gv[gv > 140]
    
    
    n_lower = lower.shape[0]
    n_upper = upper.shape[0]
    
    hypoIndex = np.sum(np.power((lltr - lower), b)) / (n_lower*d)   
    hyperIndex = np.sum(np.power((upper - ultr), a)) / (n_upper*c)
    
    igc = hypoIndex + hyperIndex
    return igc, hypoIndex, hyperIndex

In [27]:
igc(prediabetic)

(2.076534334612181, 1.5242774566473989, 0.5522568779647821)

In [259]:
#4 week

# Glycemic Lability Index
# LI is calculated based on the change in glucose level over a four-week period, so the data should be chopped into four-week basis
# REFERENCES: https://diabetes.diabetesjournals.org/content/53/4/955

def gli(x, unit = 'mmol'):
    
    time = hourAve(x)[1]
    glucose = hourAve(x)[2]
    
    if unit == 'mg':
        gv = [glu / 18 for glu in glucose]
    elif unit == 'mmol':
        gv = glucose
    else:
        print('Unit should either be mg or mmol')
    
    gli = np.sum(np.power(gv[i] - gv[i+1],2) for i in range(0, len(time)-1))
    
    return gli
    
    

In [260]:
gli(prediabetic, unit = 'mg')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


41.315957219877376

In [93]:
# The average daily risk range

#REFERENCES: https://care.diabetesjournals.org/content/29/11/2433

def adrr(x, unit):
    
    if unit == 'mg':
        gv = (x['GlucoseValue']).astype(float)
        f_bg = 1.509 *(np.power((np.log(gv)),1.084) - 5.381)
        x['f(BG)'] = f_bg
    elif unit == 'mmol':
        gv = (['GlucoseValue']).astype(float)
        f_bg = 1.509 *(np.power((np.log(gv * 18)),1.084) - 5.381)
        x['f(BG)'] = f_bg
    else:
        print('Unit should either be mg or mmol')
        
    dict_of_day = splitDay(x)
    
    adrr_lr = []
    adrr_hr = []
    hour = list(dict_of_day.keys())
    for key in dict_of_day.keys():
        f_bg = (dict_of_day.get(key)['f(BG)']).astype(float)
        rl_bg = [r_bg if r_bg < 0 else 0 for r_bg in f_bg]
        rh_bg = [r_bg if r_bg > 0 else 0 for r_bg in f_bg]
        
        rl_i = np.max(rl_bg)
        adrr_lr.append(rl_i)
        rh_i = np.max(rh_bg)
        adrr_hr.append(rh_i)
    
    lr_hr = [sum(i) for i in zip(adrr_lr, adrr_hr)] 
    adrr = np.average(lr_hr)

    
    return adrr
        

In [94]:
adrr(prediabetic, 'mg')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


0.5003907959489858

In [203]:
#Mean of daily differences

#REFERENCES: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3160264/

def modd(x, unit):
    full_day = list((splitDay(x)).items())[1:5]

    modd_list = []
    for i in range(1, len(full_day)):
        day_diff = np.abs(np.array((full_day[i][1]['GlucoseValue']).tolist(), dtype=float) - np.array((full_day[i-1][1]['GlucoseValue']).tolist(), dtype=float))
        modd_list.append(np.average(day_diff))
    
    modd = np.average(modd_list)
        
    return modd
    
    

In [204]:
modd(prediabetic, 'mg')

17.80787037037037

In [360]:
def hourAve(data):
    
    fmt_1 = '%Y-%m-%d %H:%M:%S'
    fmt_2 = '%Y-%m-%d %H'
    fmt_3 = '%Y-%m-%d'
    
    x = data
    x['Hour'] = [None] * x.shape[0]
    
    for idx in range(x.shape[0]):      
        x['Hour'].iloc[idx] = datetime.strptime(x['DisplayTime'].iloc[idx], fmt_1).strftime(fmt_2)
        
    dict_of_hour = dict(iter(x.groupby('Hour')))
    #ave_hour = [np.average((dict_of_hour[i]).get(GlucoseValue)) for i in range(len(dict_of_hour))]
    
    ave_hour = []
    
    hour = list(dict_of_hour.keys())
    for key in dict_of_hour.keys():
        ave_hour.append(np.average(((dict_of_hour.get(key))['GlucoseValue']).astype(float)))
        
    df_hour = pd.DataFrame({'DisplayTime':hour, 'GlucoseValue':ave_hour})
        
    return df_hour, hour, ave_hour
    

In [361]:
def splitDay(data):
    fmt_1 = '%Y-%m-%d %H:%M:%S'
    fmt_2 = '%Y-%m-%d %H'
    fmt_3 = '%Y-%m-%d'
    
    x = data.copy()
    x['Day'] = [None] * x.shape[0]
    for idx in range(x.shape[0]):      
        x['Day'].iloc[idx] = datetime.strptime(x['DisplayTime'].iloc[idx], fmt_1).strftime(fmt_3)
    dict_of_day = dict(iter(x.groupby('Day')))
        
    return dict_of_day